In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc


/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CPU times: user 442 ms, sys: 894 ms, total: 1.34 s
Wall time: 1.82 s


In [8]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['DOB'] = df['DOB'].fillna('11/01/82')

df['Lead_Creation_Date_ori'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date_ori'] = test['Lead_Creation_Date'].apply(helper)

df['DOB_ori'] = df['DOB'].apply(helper)
test['DOB_ori'] = test['DOB'].apply(helper)

def extract_date(d):
    tmp = strptime(d, '%d/%m/%y')
    y = tmp.tm_year
    m = tmp.tm_mon
    dy = tmp.tm_mday
    w = tmp.tm_wday
    yd = tmp.tm_yday
    dst = tmp.tm_isdst
    return '/'.join(map(str,[y,m,dy,w,yd,dst]))

df['DOB_e'] = df['DOB'].apply(extract_date)
df['Lead_Creation_Date_e'] = df['Lead_Creation_Date'].apply(extract_date)

for i in range(6):
    df['DOB_e_'+str(i)] = df['DOB_e'].apply(lambda x: int(x.split('/')[i]))
    df['Lead_Creation_Date_e'+str(i)] = df['Lead_Creation_Date_e'].apply(lambda x: int(x.split('/')[i]))

_ = df.pop('DOB_e')
_ = df.pop('Lead_Creation_Date_e')
_ = df.pop('DOB')
_ = df.pop('Lead_Creation_Date')

(69713, 22)
(30037, 21)
CPU times: user 6.96 s, sys: 244 ms, total: 7.2 s
Wall time: 7.19 s


In [3]:
df['Cat']=0
test['Cat']=1
comb = pd.concat([df, test])
comb.shape

for i in comb.select_dtypes('object'):
    comb[i] = pd.factorize(comb[i])[0]

train, test = comb[comb['Cat']==0], comb[comb['Cat']==1]
_, _ = train.pop('Cat'), test.pop('Cat')
print train.shape, test.shape

(69713, 34) (30037, 34)


In [10]:
df.head()

,Gender,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,...,DOB_e_1,Lead_Creation_Date_e1,DOB_e_2,Lead_Creation_Date_e2,DOB_e_3,Lead_Creation_Date_e3,DOB_e_4,Lead_Creation_Date_e4,DOB_e_5,Lead_Creation_Date_e5
0,Female,C10001,A,COM0044082,A,4.0,2000.0,B001,P,N,...,7,7,23,15,0,4,204,197,-1,-1
1,Male,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,...,12,7,7,4,6,0,341,186,-1,-1
2,Male,C10125,C,COM0005267,C,4.0,2250.0,B003,G,Y,...,12,7,10,19,4,1,344,201,-1,-1
3,Male,C10477,C,COM0004143,A,4.0,3500.0,B003,G,Y,...,1,7,30,9,0,5,30,191,-1,-1
4,Male,C10002,A,COM0001781,A,4.0,10000.0,B001,P,Y,...,4,7,19,20,4,2,109,202,-1,-1


In [11]:
pd.factorize(df['City_Code'])

(array([0, 1, 2, ..., 6, 4, 1]),
 Index([u'C10001', u'C10003', u'C10125', u'C10477', u'C10002', u'C10402',
        u'C10022', u'C10014', u'C10030', u'C10032',
        ...
        u'C10692', u'C10563', u'C10612', u'C10717', u'C10626', u'C10693',
        u'C10673', u'C10685', u'C10676', u'C10686'],
       dtype='object', length=678))

In [4]:
train = train.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)


s = StandardScaler()
train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape

(69713, 34) (30037, 34)


### train test split

In [9]:
%%time
m = lambda *x: [i.shape for i in x]
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.2)
print m(x_train, y_train, x_test, y_test)


[(55770, 34), (55770,), (13943, 34), (13943,)]
CPU times: user 29 ms, sys: 2 ms, total: 31 ms
Wall time: 30.2 ms


### create xgb matrix

In [10]:
%%time
d_train = xgb.DMatrix(x_train, label=y_train)
d_test = xgb.DMatrix(x_test, label=y_test)


CPU times: user 39 ms, sys: 2 ms, total: 41 ms
Wall time: 39.3 ms


In [18]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 9,
    'subsample': 0.6,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.0005,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [19]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200, 
                verbose_eval=50,  maximize=True,
               )
                

'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.741896	test-auc:0.744976
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[50]	train-auc:0.807287	test-auc:0.813703
[100]	train-auc:0.811757	test-auc:0.818746
[150]	train-auc:0.814606	test-auc:0.818851
[200]	train-auc:0.816373	test-auc:0.819496
[250]	train-auc:0.817684	test-auc:0.819149
[300]	train-auc:0.818406	test-auc:0.818774
[350]	train-auc:0.819913	test-auc:0.81948
[400]	train-auc:0.821143	test-auc:0.819694
[450]	train-auc:0.835136	test-auc:0.829827
[500]	train-auc:0.835259	test-auc:0.829089
[550]	train-auc:0.836313	test-auc:0.829164
[600]	train-auc:0.837219	test-auc:0.829839
[650]	train-auc:0.838108	test-auc:0.830103
[700]	train-auc:0.839221	test-auc:0.830351
[750]	train-auc:0.840935	test-auc:0.830776
[800]	train-auc:0.841942	test-auc:0.831188
[850]	train-auc:0.844533	test-auc:0.832784
[900]	train-auc:0.845358	test-auc:0.833111
[950]	train-auc:0.84628	test-auc:0.83329
[100

'!!! used 17.88 s'

In [21]:
clf.get_fscore()

{'f0': 67,
 'f1': 703,
 'f11': 1836,
 'f12': 1795,
 'f13': 319,
 'f14': 411,
 'f15': 2242,
 'f16': 3488,
 'f17': 88,
 'f18': 1084,
 'f2': 15,
 'f21': 110,
 'f22': 831,
 'f23': 398,
 'f24': 777,
 'f26': 184,
 'f27': 854,
 'f28': 202,
 'f29': 2092,
 'f3': 1076,
 'f30': 164,
 'f31': 540,
 'f32': 30,
 'f33': 583,
 'f5': 604,
 'f6': 422,
 'f7': 812,
 'f8': 503,
 'f9': 1373}

In [66]:
clf.save_rabit_checkpoint?

In [45]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 13.9 s, sys: 6.85 s, total: 20.7 s
Wall time: 20.7 s


In [46]:
%%time
predb = clf.predict(dtest)


CPU times: user 4.31 s, sys: 3 ms, total: 4.31 s
Wall time: 268 ms


___

In [47]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [48]:
submit.to_csv('xgb-ext-date.csv',index=False)

In [22]:
clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba